In [7]:
from astropy.io import fits

file_path = 'DES-Y3_xipm_and_KiDS-1000_COSEBIs_2.0_300.0.fits'

with fits.open(file_path) as hdul:
    # hdul.info()
    xip_data = hdul['xip'].data
    xim_data = hdul['xim'].data
    desy3_covmat = hdul['COVMAT'].data

xi_minus_data = xim_data['VALUE']
xi_minus_ang = xim_data['ANG']
xi_plus_data = xip_data['VALUE']
xi_plus_ang = xip_data['ANG']


In [19]:
import numpy as np
from numba import njit

nb_l = 1

def derivative(x:float, l:int, n:int):
    result = (x**2 + 1)**l
    for _ in range(n):
        result = 2 * l * x * (x**2 + 1)**(l - 1)
        l -= 1
    return result

def richardson_extrapolation(f, x, l, n, h=1e-5):
    def derivative_order_n(f, x, l, n, h):
        result = 0
        for k in range(2*n + 1):
            coeff = (-1)**k * np.math.comb(2*n, k)
            result += coeff * f(x + (k - n) * h, l)
        return result / (h**n)

    D1 = derivative_order_n(f, x, l, n, h)
    D2 = derivative_order_n(f, x, l, n, h / 2)
    if n==0:
        return 0
    return (4**n * D2 - D1) / (4**n - 1)

def f(x:float, l:int):
    return (x**2 + 1)**l

def P(x:float, l:int, m:int):
    return (1-x**2)/(2**l * np.math.factorial(l)) * richardson_extrapolation(f, x, l, l+m)

def Gp(x:float, l:int, m:int=2):
    return - ( (l-m**2) / (1-x**2) + 0.5 * l * (l-1) ) * P(x, l, m) + (l+m) * (x / (1-x**2)) * P(x, l-1, m)

def Gm(theta:float):
    return m * ( (l-1) * x / (1-x**2) * P(x, l, m) - (l+m) * (1 / (1-x**2)) * P(x, l-1, m))

def ClEE(l:int, i:int, j:int):
    return 0.0

def ClBB(l:int, i:int, j:int):
    return 0.0

def xip(theta:float, i:int, j:int):
    xi = 0.0
    for l in range(nb_l):
        xi += (2*l + 1)/(2*np.pi*(l+1)**2) * (Gp(np.cos(theta)) + Gm(np.cos(theta))) * (ClEE(l, i, j) + ClBB(l, i, j))
    return xi


In [30]:
P(1,10,10)

/tmp/ipykernel_3430/548842123.py:31: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  return (1-x**2)/(2**l * np.math.factorial(l)) * richardson_extrapolation(f, x, l, l+m)
/tmp/ipykernel_3430/548842123.py:17: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  coeff = (-1)**k * np.math.comb(2*n, k)


0.0